In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import math

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import metrics

In [2]:
#importing the dataset 
dataset = pd.read_csv('diabetic_data.csv')
dataset = dataset.drop(columns=['encounter_id','patient_nbr'])
dataset = dataset.replace('<30',1).replace('>30',0).replace('NO',0)

In [ ]:
grouped = dataset.groupby('readmitted')
grouped.hist(figsize=(12,12))
print("Size",grouped.size())


In [3]:
dataset = dataset.replace('?', np.NaN)
dataset.isnull().sum()

race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [12]:
label = dataset.filter(items=['readmitted'])
train = dataset.drop(columns=['readmitted'])
print(train.columns)

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed'],
      dtype='object')


In [5]:
print(label)
print(train)

readmitted
0                0
1                0
2                0
3                0
4                0
...            ...
101761           0
101762           0
101763           0
101764           0
101765           0

[101766 rows x 1 columns]
                   race  gender      age weight  admission_type_id  \
0             Caucasian  Female   [0-10)    NaN                  6   
1             Caucasian  Female  [10-20)    NaN                  1   
2       AfricanAmerican  Female  [20-30)    NaN                  1   
3             Caucasian    Male  [30-40)    NaN                  1   
4             Caucasian    Male  [40-50)    NaN                  1   
...                 ...     ...      ...    ...                ...   
101761  AfricanAmerican    Male  [70-80)    NaN                  1   
101762  AfricanAmerican  Female  [80-90)    NaN                  1   
101763        Caucasian    Male  [70-80)    NaN                  1   
101764        Caucasian  Female  [80-90)    NaN      

In [7]:
from sklearn.preprocessing import LabelEncoder
encoded_label = LabelEncoder().fit_transform(label)
print(encoded_label)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
numeric_features = [ 
         'time_in_hospital','num_lab_procedures','num_procedures',
          'num_medications', 'number_outpatient','number_emergency',
           'number_inpatient', 'diag_1', 'diag_2', 'diag_3','number_diagnoses',  
       ]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['race', 'gender', 'age', 'weight','admission_type_id',
                        'discharge_disposition_id','admission_source_id',
                        'payer_code', 'medical_specialty','max_glu_serum',
                        'A1Cresult', 'metformin','repaglinide', 'nateglinide',
                        'chlorpropamide', 'glimepiride','acetohexamide', 
                        'glipizide', 'glyburide', 'tolbutamide','pioglitazone', 
                        'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
                        'tolazamide', 'examide', 'citoglipton', 'insulin',
                        'glyburide-metformin', 'glipizide-metformin',
                        'glimepiride-pioglitazone', 'metformin-rosiglitazone',
                        'metformin-pioglitazone', 'change', 'diabetesMed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe.fit(X_train, y_train)
print("model score: %.3f" % pipe.score(X_test, y_test))